## Drop-OFF Analysis

In [33]:
import pandas as pd

### Data Loading & Overview

In [34]:
pageviews = pd.read_csv(r"D:\Data Analysis Learning\Portfolio Projects\toy-store-ecommerce-analysis\Toy Store E-Commerce Database\website_pageviews.csv")

In [35]:
pageviews.sort_values('website_session_id').head(100)

,website_pageview_id,created_at,website_session_id,pageview_url
0,1,2012-03-19 08:04:16,1,/home
1,2,2012-03-19 08:16:49,2,/home
2,3,2012-03-19 08:26:55,3,/home
3,4,2012-03-19 08:37:33,4,/home
4,5,2012-03-19 09:00:55,5,/home
...,...,...,...,...
96,97,2012-03-19 14:14:07,54,/products
98,99,2012-03-19 14:18:16,54,/the-original-mr-fuzzy
95,96,2012-03-19 14:12:09,54,/home
97,98,2012-03-19 14:15:08,55,/home


### Drop-off Rate Calculation

#### Where do customers drop off in the purchase funnel?

In [36]:

last_page = pd.DataFrame(pageviews.groupby('website_session_id')['pageview_url'].last())

In [37]:
drop_off = pd.DataFrame(last_page['pageview_url'].value_counts())
drop_off['pageview_url'] = drop_off.index

In [38]:
drop_off.index = range(len(drop_off.index))

In [39]:
drop_off = drop_off[['pageview_url', 'count']]


In [40]:
drop_off.rename(columns={'count' : 'drop_off_count'})

,pageview_url,drop_off_count
0,/the-original-mr-fuzzy,92568
1,/lander-2,59249
2,/home,57346
3,/products,51017
4,/lander-3,39733
5,/thank-you-for-your-order,32313
6,/cart,30469
7,/lander-1,25330
8,/lander-5,25131
9,/billing-2,17748


#### How many times each page in the funnel is visited?

In [41]:
page_enterance = pd.DataFrame(pageviews.groupby('pageview_url')['website_session_id'].count())
page_enterance['pageview_url'] = page_enterance.index

In [42]:
page_enterance.index = range(len(page_enterance.index))
page_enterance = page_enterance[['pageview_url','website_session_id']]

In [43]:
page_enterance

,pageview_url,website_session_id
0,/billing,3617
1,/billing-2,48441
2,/cart,94953
3,/home,137576
4,/lander-1,47574
5,/lander-2,131170
6,/lander-3,79000
7,/lander-4,9385
8,/lander-5,68166
9,/products,261231


#### Mergin optained results to a single dataframe

In [44]:
drop_off_rate = pd.merge(page_enterance, drop_off, left_on='pageview_url', right_on='pageview_url', how='left')
drop_off_rate.columns = ['pageview_url','page_enterance_count', 'drop_off_count']
drop_off_rate

,pageview_url,page_enterance_count,drop_off_count
0,/billing,3617,1997
1,/billing-2,48441,17748
2,/cart,94953,30469
3,/home,137576,57346
4,/lander-1,47574,25330
5,/lander-2,131170,59249
6,/lander-3,79000,39733
7,/lander-4,9385,4851
8,/lander-5,68166,25131
9,/products,261231,51017


#### Calculating Drop-off Rates on page visits basis and on total sessions basis

In [45]:
drop_off_rate['drop_off_rate'] = drop_off_rate['drop_off_count'] * 100 / drop_off_rate['page_enterance_count']
drop_off_rate['drop_off_rate_of_total'] = drop_off_rate['drop_off_count'] * 100 / drop_off_rate['drop_off_count'].sum()
drop_off_rate[['drop_off_rate','drop_off_rate_of_total']] = round(drop_off_rate[['drop_off_rate','drop_off_rate_of_total']], 2)


In [46]:
drop_off_rate.sort_values(by='drop_off_rate_of_total', ascending=False)

,pageview_url,page_enterance_count,drop_off_count,drop_off_rate,drop_off_rate_of_total
15,/the-original-mr-fuzzy,162525,92568,56.96,19.58
5,/lander-2,131170,59249,45.17,12.53
3,/home,137576,57346,41.68,12.13
9,/products,261231,51017,19.53,10.79
6,/lander-3,79000,39733,50.29,8.40
11,/thank-you-for-your-order,32313,32313,100.00,6.83
2,/cart,94953,30469,32.09,6.44
4,/lander-1,47574,25330,53.24,5.36
8,/lander-5,68166,25131,36.87,5.31
1,/billing-2,48441,17748,36.64,3.75
